<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 0.Imports

In [32]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")
print('finished')

finished


## <span style='color:#ff5f27'> 🌍 1.1 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [33]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
with open('../../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
#Now we just need to modify the first nb. The other ones are automatically updated
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1150100
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "italy", "city": "toscana", "street": "pisa", "aqicn_url": "https://api.waqi.info/feed/@9432", "latitude": 43.47, "longitude": 10.68}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [34]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
weather_fg

---

## <span style='color:#ff5f27'> 2.Retrieve Datas using APIs</span>


### <span style='color:#ff5f27'> 2.1 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [35]:
#import requests
#import pandas as pd
#yesterday = today - datetime.timedelta(days=3)

#aq_tmp_df = util.get_pm25(aqicn_url, country, city, street, yesterday, AQI_API_KEY)
# Function that gives back df from API request 
#aq_tmp_df

In [36]:
#fin_df = pd.concat([fin_df,aq_tmp_df])

#fin_df

In [37]:
#air_quality_fg.insert(existing_df)

In [38]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,95.0,italy,toscana,pisa,2024-11-18,https://api.waqi.info/feed/@9432


In [39]:
#aq_today_df.info()

In [40]:
dfa = air_quality_fg.read()
dfa = dfa.sort_values(by='date', ascending=False)  # Sorts in ascending order
dfa

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.28s) 


,date,pm25,country,city,street,url
2748,2024-11-18 00:00:00+00:00,95.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
2755,2024-11-17 00:00:00+00:00,95.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
2756,2024-11-16 00:00:00+00:00,95.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
2754,2024-11-15 00:00:00+00:00,95.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
2752,2024-11-14 00:00:00+00:00,59.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
...,...,...,...,...,...,...
1403,2017-01-07 00:00:00+00:00,55.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
1998,2017-01-06 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
1331,2017-01-05 00:00:00+00:00,50.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432
730,2017-01-04 00:00:00+00:00,82.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432


### <span style='color:#ff5f27'> 2.2 Get Weather Forecast data</span>

In [41]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
# Function that gives back df from API request 
hourly_df.head()

Coordinates 43.5°N 10.75°E
Elevation 147.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
0,2024-11-18 00:00:00,9.85,0.0,9.585739,145.713074
1,2024-11-18 01:00:00,9.60,0.0,8.714677,141.709793
2,2024-11-18 02:00:00,9.40,0.0,7.928177,140.527573
3,2024-11-18 03:00:00,9.35,0.0,6.877789,137.121109
4,2024-11-18 04:00:00,9.50,0.0,5.600286,135.000107


In [42]:
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df.head()

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-18,16.40,0.2,4.104631,164.744827,toscana
1,2024-11-19,14.80,0.5,14.336861,231.115494,toscana
2,2024-11-20,16.10,0.1,33.166920,242.878769,toscana
3,2024-11-21,11.85,0.0,9.693296,158.198532,toscana
4,2024-11-22,13.05,0.7,27.381306,298.249634,toscana


In [43]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">3.3 ⬆️ Uploading new data to the Feature Store</span>

In [44]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-18 21:50:51,117 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1150100/fs/1140803/fg/1354080


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1150100/jobs/named/air_quality_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1a2e5623640>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 677934
         }
       },
       "result": {
         "observed_value": 95.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-18T08:50:51.000117Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsu

In [45]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-18 21:51:00,142 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1150100/fs/1140803/fg/1354080


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1150100/jobs/named/air_quality_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1a2e5623640>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 677934
         }
       },
       "result": {
         "observed_value": 95.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-18T08:51:00.000141Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsu

In [46]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-18 21:51:09,580 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1150100/fs/1140803/fg/1345889


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1150100/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1a2e21cc220>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 669710
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-18T08:51:09.000579Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betwe

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store


In [47]:
dfa['pm25_rolling'] = dfa['pm25'].rolling(window=3, min_periods=1).mean()
dfa['pm25_rolling'] = dfa['pm25_rolling'].fillna(dfa['pm25'])
dfa

,date,pm25,country,city,street,url,pm25_rolling
2748,2024-11-18 00:00:00+00:00,95.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,95.000000
2755,2024-11-17 00:00:00+00:00,95.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,95.000000
2756,2024-11-16 00:00:00+00:00,95.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,95.000000
2754,2024-11-15 00:00:00+00:00,95.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,95.000000
2752,2024-11-14 00:00:00+00:00,59.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,83.000000
...,...,...,...,...,...,...,...
1403,2017-01-07 00:00:00+00:00,55.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,81.000000
1998,2017-01-06 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,57.666667
1331,2017-01-05 00:00:00+00:00,50.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,49.000000
730,2017-01-04 00:00:00+00:00,82.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,58.000000


## Adding the Rolling Windows Datagroup

In [48]:
air_quality_roll_fg = fs.get_or_create_feature_group(
    name='air_quality_roll',
    description='Air Quality characteristics of each day, rolling feature',
    version=1,
    primary_key=['city', 'street', 'date'],
    event_time="date"
)

In [49]:
air_quality_roll_fg.insert(dfa)

Uploading Dataframe: 0.00% |          | Rows 0/2757 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_roll_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1150100/jobs/named/air_quality_roll_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1a2e3631a80>, None)

In [50]:
df_lagged = air_quality_roll_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.16s) 


In [51]:
df_lagged

,date,pm25,country,city,street,url,pm25_rolling
0,2021-01-15 00:00:00+00:00,34.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,55.000000
1,2024-06-29 00:00:00+00:00,55.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,39.666667
2,2022-06-11 00:00:00+00:00,30.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,30.000000
3,2019-03-07 00:00:00+00:00,42.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,39.333333
4,2023-11-05 00:00:00+00:00,21.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,19.666667
...,...,...,...,...,...,...,...
2752,2022-02-25 00:00:00+00:00,57.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,36.000000
2753,2018-11-06 00:00:00+00:00,25.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,33.333333
2754,2024-04-20 00:00:00+00:00,25.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,21.000000
2755,2022-05-08 00:00:00+00:00,34.0,italy,toscana,pisa,https://api.waqi.info/feed/@9432,46.333333
